<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->

# <font color='blue'>Data Science Academy</font>

# <font color='blue'>Processamento de Linguagem Natural com Transformers</font>

## <font color='blue'>Lab 7</font>

## <font color='blue'>Ajuste Fino de LLM Open-Source Com Dados da Área de Direito</font>

## Instalando e Carregando Pacotes

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.10.12


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# Instalando os pacotes
!pip install -q -r requirements.txt

In [4]:
# Imports
import logging
import shutil
import nltk
import evaluate
import transformers
import datasets
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

logging     : 0.5.1.2
datasets    : 2.17.0
nltk        : 3.8.1
evaluate    : 0.4.1
numpy       : 1.25.2
transformers: 4.37.2



In [6]:
# Remove as pastas, se já existirem. Isso evita erros ao executar o jupyter a partir da segunda vez em diante
try:
    shutil.rmtree('logs_treino')
    shutil.rmtree('resultados_treino')
    shutil.rmtree('modelo_salvo')
except:
    print('As pastas não existem ou já foram removidas!')

As pastas não existem ou já foram removidas!


## Carregando os Dados

In [7]:
# Define o nome do dataset
nome_dataset = "dataset.csv"

In [8]:
# Carrega os dados
dsa_dataset = load_dataset('csv', data_files = nome_dataset, delimiter = ',')

Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
# Divisão em treino e teste com proporção 80/20
dsa_dataset = dsa_dataset["train"].train_test_split(test_size = 0.2)

In [10]:
# Visualiza o formato do dataset
dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer'],
        num_rows: 749
    })
})

## Baixando Tokenizador e Modelo Para o Processo de Tokenização

https://huggingface.co/google/flan-t5-base

In [11]:
# Carrega o tokenizador
tokenizador = T5Tokenizer.from_pretrained("google/flan-t5-base", legacy=False)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
# Visualiza o tokenizador
tokenizador

T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>', '

In [13]:
# Carrega o LLM pré-treinado
modelo = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [14]:
# Visualiza a arquitetura do LLM
modelo

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [15]:
# Data collator para concatenar tokenizador e modelo
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizador, model = modelo)

In [16]:
# Visualiza o data collator
data_collator

DataCollatorForSeq2Seq(tokenizer=T5Tokenizer(name_or_path='google/flan-t5-base', vocab_size=32000, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<

## Pré-Processamento

In [17]:
# Todos os inputs vão receber como prefixo: "answer the question" (responda a questão)
prefix = "answer the question: "

In [18]:
# Função de pré-processamento
def dsa_fn_preprocess(data):

    # Concatena o prefixo a cada pergunta na lista de perguntas fornecida em data["question"]
    inputs = [prefix + doc for doc in data["question"]]

    # Usa o tokenizer para converter as perguntas processadas em tokens com um comprimento máximo de 128,
    # truncando as que forem mais longas
    model_inputs = tokenizador(inputs, max_length = 128, truncation = True)

    # Tokeniza as respostas fornecidas em data["answer"] com um comprimento máximo de 512,
    # truncando as que forem mais longas
    labels = tokenizador(text_target = data["answer"], max_length = 512, truncation = True)

    # Adiciona os tokens das respostas como rótulos no dicionário de entradas do modelo
    model_inputs["labels"] = labels["input_ids"]

    # Retorna o dicionário processado contendo tanto as entradas tokenizadas quanto os rótulos
    return model_inputs

In [19]:
# Aplica a função de pré-processamento ao dataset gerando o dataset tokenizado
tokenized_dsa_dataset = dsa_dataset.map(dsa_fn_preprocess, batched = True)

Map:   0%|          | 0/2993 [00:00<?, ? examples/s]

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

In [20]:
# Visualiza o dataset tokenizado
tokenized_dsa_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2993
    })
    test: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 749
    })
})

In [21]:
tokenized_dsa_dataset['train']['question'][0]

'Q: City passed a new ordinance a year after I have owned chickens and are saying I have to get rid of them am I grandfather. I’ve lived there for three years and owned chickens they whole three years and are now telling me I have to get rid of them new ordinance passed may 2nd this year '

In [22]:
tokenized_dsa_dataset['train']['answer'][0]

'A:One would need to see the ordinance and it’s history to properly answer this question.'

In [23]:
tokenized_dsa_dataset['train']['input_ids'][0]

[1525,
 8,
 822,
 10,
 1593,
 10,
 896,
 2804,
 3,
 9,
 126,
 24417,
 3,
 9,
 215,
 227,
 27,
 43,
 4157,
 3832,
 7,
 11,
 33,
 2145,
 27,
 43,
 12,
 129,
 5413,
 13,
 135,
 183,
 27,
 18573,
 5,
 27,
 22,
 162,
 4114,
 132,
 21,
 386,
 203,
 11,
 4157,
 3832,
 7,
 79,
 829,
 386,
 203,
 11,
 33,
 230,
 5188,
 140,
 27,
 43,
 12,
 129,
 5413,
 13,
 135,
 126,
 24417,
 2804,
 164,
 204,
 727,
 48,
 215,
 1]

In [24]:
tokenized_dsa_dataset['train']['labels'][0]

[71,
 10,
 10723,
 133,
 174,
 12,
 217,
 8,
 24417,
 11,
 34,
 22,
 7,
 892,
 12,
 3085,
 1525,
 48,
 822,
 5,
 1]

## Definindo a Métrica de Avaliação do Modelo

In [25]:
# O pacote "punkt" é específico para a tarefa de tokenização, que envolve a divisão de um texto
# em uma lista de sentenças
nltk.download("punkt", quiet = True)

True

In [26]:
# Define a métrica
metric = evaluate.load("rouge")

ROUGE é uma família de métricas usada para avaliar a qualidade de resumos de texto ao comparar um resumo gerado automaticamente com um ou mais resumos de referência feitos por humanos. ROUGE mede a sobreposição de n-gramas, sequências de palavras e/ou subsequências entre o texto gerado e o texto de referência, fornecendo medidas como precisão, recall e score F1, dependendo da variante da métrica ROUGE utilizada (por exemplo, ROUGE-N, ROUGE-L, ROUGE-W, etc.).

In [27]:
# Função de cálculo das métricas
def dsa_calcula_metricas(eval_preds):

    # Desempacota as predições e os rótulos do argumento eval_preds
    previsoes, labels = eval_preds

    # Substitui todos os valores diferentes de -100 em labels pelo ID do token de preenchimento
    labels = np.where(labels != -100,
                      labels,
                      tokenizador.pad_token_id)

    # Decodifica as previsões para texto, ignorando tokens especiais
    previsoes_decodificadas = tokenizador.batch_decode(previsoes,
                                                       skip_special_tokens = True)

    # Decodifica os labels para texto, ignorando tokens especiais
    labels_decodificados = tokenizador.batch_decode(labels,
                                                    skip_special_tokens = True)

    # Adiciona uma nova linha após cada sentença para as previsões decodificadas,
    # preparando-as para a avaliação ROUGE
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in previsoes_decodificadas]

    # Adiciona uma nova linha após cada label para as previsões decodificadas,
    # preparando-os para a avaliação ROUGE
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in labels_decodificados]

    # Calcula a métrica ROUGE entre as previções e os rótulos decodificados, utilizando um stemmer
    resultado = metric.compute(predictions = previsoes_decodificadas,
                               references = labels_decodificados,
                               use_stemmer = True)

    # Retorna o resultado da métrica ROUGE
    return resultado

In [28]:
# Define argumentos de treino
dsa_training_args = Seq2SeqTrainingArguments(output_dir = "resultados_treino",
                                             evaluation_strategy = "epoch",
                                             learning_rate = 3e-4,
                                             logging_dir = "logs_treino",
                                             logging_steps = 1,
                                             per_device_train_batch_size = 4,
                                             per_device_eval_batch_size = 2,
                                             weight_decay = 0.01,
                                             save_total_limit = 3,
                                             num_train_epochs = 3,
                                             predict_with_generate = True,
                                             push_to_hub = False)

In [29]:
# Define o trainer
dsa_trainer = Seq2SeqTrainer(model = modelo,
                             args = dsa_training_args,
                             train_dataset = tokenized_dsa_dataset["train"],
                             eval_dataset = tokenized_dsa_dataset["test"],
                             tokenizer = tokenizador,
                             data_collator = data_collator,
                             compute_metrics = dsa_calcula_metricas)

## Treinamento do Modelo

In [30]:
%%time
dsa_trainer.train()

CPU times: user 15min 50s, sys: 1min, total: 16min 50s
Wall time: 17min 9s


TrainOutput(global_step=2247, training_loss=2.467178409981314, metrics={'train_runtime': 1029.2083, 'train_samples_per_second': 8.724, 'train_steps_per_second': 2.183, 'total_flos': 1471587685161984.0, 'train_loss': 2.467178409981314, 'epoch': 3.0})

In [31]:
# Salva o modelo
dsa_trainer.save_model('modelo_salvo')

**ROUGE-1** mede a sobreposição de unigramas (palavras individuais).

**ROUGE-2** mede a sobreposição de bigramas (pares de palavras consecutivas).

**ROUGE-L** mede a sobreposição da subsequência mais longa comum entre o resumo gerado e o resumo de referência. Isso leva em conta a ordem das palavras, mas permite lacunas. Valores mais altos indicam melhor desempenho. ROUGE-L é calculado com base na similaridade entre as sequências, levando em consideração a precisão, o recall e a média harmônica entre eles.

Valores mais altos de ROUGE indicam uma maior semelhança entre o resumo gerado e o resumo de referência, o que geralmente é interpretado como uma indicação de melhor qualidade do resumo. No entanto, é importante lembrar que nenhuma métrica pode capturar completamente a qualidade de um resumo e é útil complementar a avaliação com análises qualitativas ou outras métricas.

## Deploy e Uso do Modelo

In [32]:
# Carrega o tokenizador final salvo em disco
tokenizador_dsa_final = AutoTokenizer.from_pretrained('modelo_salvo')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [33]:
# Carrega o modelo final salvo em disco
modelo_dsa_final = AutoModelForSeq2SeqLM.from_pretrained('modelo_salvo')

In [34]:
# Visualiza a arquitetura do modelo
modelo_dsa_final

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [35]:
# Texto de entrada
texto_entrada = "Should one wait until the offender is released from prison before filing a protection order"

In [36]:
# Tokeniza o input
texto_entrada_tokenizado = tokenizador_dsa_final(texto_entrada, return_tensors = "pt").input_ids

In [37]:
# Gera a saída (previsão do modelo)
texto_saida_tokenizado = modelo_dsa_final.generate(texto_entrada_tokenizado,
                                                   max_length = 50,
                                                   temperature = 0.4,
                                                   do_sample = True)

In [38]:
texto_saida_tokenizado

tensor([[    0,    71,  1711,   455,    19,     3,     9,  1281,   455,    24,
          1250,     3,     9, 11819,    12,    36,  1883,    45,  5714,   274,
          9479,     3,     9, 13310,   455,     5,    37,   455,    19,     3,
             9,  1281,   455,    24,     3,     9, 11819,   398,  1716,   274,
           271,  1883,    45,  5714,     5,    37, 11819,   398,  1716,     3]])

In [39]:
# Decode da saída
texto_saida = tokenizador_dsa_final.decode(texto_saida_tokenizado[0],
                                           skip_special_tokens = True)

In [40]:
# Resultado
print("Pergunta:", texto_entrada)
print("Resposta:", texto_saida)

Pergunta: Should one wait until the offender is released from prison before filing a protection order
Resposta: A protection order is a legal order that allows a defendant to be released from prison before filing a protective order. The order is a legal order that a defendant must serve before being released from prison. The defendant must serve 


# Fim